<a href="https://colab.research.google.com/github/kaushik912/blogger_pages/blob/main/RAG_on_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RAG on Code

In [1]:
import os

In [2]:
!pip install gitpython langchain openai chromadb tiktoken langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 940.5 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:0

In [1]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

Clone Github repositories

In [2]:
%pwd

'/content'

In [3]:
!mkdir test_repo

In [4]:
repo_path = "test_repo/"

Repo.clone_from("https://github.com/kshitijkutumbe/Visa-Sanction-Prediction-ML.git", to_path=repo_path)

<git.repo.base.Repo '/content/test_repo/.git'>

In [5]:
repo_path = "test_repo/"

loader = GenericLoader.from_filesystem(repo_path+'/us_visa_prediction/pipeline',
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

In [6]:
documents = loader.load()

In [ ]:
documents

Chunkings

In [8]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [9]:
texts = documents_splitter.split_documents(documents)

In [10]:
len(texts)

9

In [11]:
texts[0]

Document(metadata={'source': 'test_repo/us_visa_prediction/pipeline/training_pipeline.py', 'language': <Language.PYTHON: 'python'>}, page_content='import sys\nfrom us_visa_prediction.exception import USvisaException\nfrom us_visa_prediction.logger import logging\n\nfrom us_visa_prediction.components.data_ingestion import DataIngestion\nfrom us_visa_prediction.components.data_validation import DataValidation\nfrom us_visa_prediction.components.data_transformation import DataTransformation\nfrom us_visa_prediction.components.model_trainer import ModelTrainer\nfrom us_visa_prediction.components.model_evaluation import ModelEvaluation\nfrom us_visa_prediction.components.model_pusher import ModelPusher\n\nfrom us_visa_prediction.entity.config_entity import (DataIngestionConfig,\n                                          DataValidationConfig,\n                                          DataTransformationConfig,\n                                          ModelTrainerConfig,\n                  

Embedding model

In [12]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [13]:
embeddings=OpenAIEmbeddings(disallowed_special=())

<ipython-input-13-c134ac42d581>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings=OpenAIEmbeddings(disallowed_special=())


Knowledge base (vector DB)

In [14]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='./data')
vectordb.persist()

<ipython-input-14-89dfd31ceb8b>:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


LLM Wrapper

In [15]:
llm = ChatOpenAI()

<ipython-input-15-1f2ff8f9afda>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI()


In [16]:
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

<ipython-input-16-a00c93c5d0bc>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)


In [17]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

Q&A

In [18]:
question = "what is happening in training pipeline?"

In [19]:
result = qa(question)
print(result['answer'])

<ipython-input-19-e1db7fe6eb4b>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa(question)


In the provided code, the training pipeline is running through a series of steps to train a model. Here's a breakdown of the process:

1. **Data Ingestion**: The pipeline starts with data ingestion, where data is collected and ingested into the pipeline.
2. **Data Validation**: Next, the data goes through a validation step to ensure its quality and integrity.
3. **Data Transformation**: The data then undergoes transformation to prepare it for model training.
4. **Model Training**: The transformed data is used to train a model.
5. **Model Evaluation**: The trained model is evaluated to assess its performance.
6. **Model Acceptance Check**: If the model meets certain criteria for acceptance, it proceeds to the next step. Otherwise, a message is logged saying the model was not accepted.
7. **Model Pushing**: Finally, if the model is accepted, it is pushed to a destination using the model pusher.

Each step in the pipeline is handled by a specific method within the `TrainPipeline` class, s